In [1]:
# Import the required variables
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

In [3]:
def num_plot(df, col, title, symb):
    fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8,5),gridspec_kw={"height_ratios": (.2, .8)})
    ax[0].set_title(title,fontsize=18)
    sns.boxplot(x=col, data=df, ax=ax[0])
    ax[0].set(yticks=[])
    sns.histplot(x=col, data=df, ax=ax[1])
    ax[1].set_xlabel(col, fontsize=16)
    plt.axvline(df[col].mean(), color='darkgreen', linewidth=2.2, label='mean=' + str(np.round(df[col].mean(),1)) + symb)
    plt.axvline(df[col].median(), color='red', linewidth=2.2, label='median='+ str(np.round(df[col].median(),1)) + symb)
    plt.axvline(df[col].mode()[0], color='purple', linewidth=2.2, label='mode='+ str(df[col].mode()[0]) + symb)
    plt.legend(bbox_to_anchor=(1, 1.03), ncol=1, fontsize=17, fancybox=True, shadow=True, frameon=True)
    plt.tight_layout()
    plt.show()

In [4]:
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlesize=14, titlepad=10)

<Figure size 432x288 with 0 Axes>

In [8]:
# Import the required variables
import pandas as pd
import numpy as np

# Read the data
Res_Nonhydro_Capita = pd.read_excel ("../data/processed/Res_Nonhydro_Capita.xlsx", index_col = 0)
SDGs_Ranks_2021 = pd.read_excel ("../data/processed/SDGs_Ranks_2021.xlsx", index_col = 0)
SDGs_Scores_2021 = pd.read_excel ("../data/processed/SDGs_Scores_2021.xlsx", index_col = 0)

# Only consider the 2021 year
Res_Nonhydro_Capita_2021 = Res_Nonhydro_Capita[[2021]]
Res_Nonhydro_Capita_2021= Res_Nonhydro_Capita_2021.rename(columns={ 2021: 'Res_Nonhydro_Capita_2021'})

# Do not consider the first column
SDGs_Ranks_2021 = SDGs_Ranks_2021.iloc[:,1:]
SDGs_Scores_2021 = SDGs_Scores_2021.iloc[:,1:]

# Append the target to the Datasets
SDGs_Ranks_2021 = pd.concat([SDGs_Ranks_2021, Res_Nonhydro_Capita_2021], axis=1)
SDGs_Scores_2021 = pd.concat([SDGs_Scores_2021, Res_Nonhydro_Capita_2021], axis=1)

# Dicotomizing Res_Nonhydro_Capita_2021 in Wealthy and Unwealthy Countries
median = Res_Nonhydro_Capita_2021['Res_Nonhydro_Capita_2021'].median()
Res_Nonhydro_Capita_2021['Res_Nonhydro_Capita_2021'] = Res_Nonhydro_Capita_2021.Res_Nonhydro_Capita_2021.map(lambda x: 1 if x >= median else 0)
Res_Nonhydro_Capita_2021 = Res_Nonhydro_Capita_2021.rename(columns={'Res_Nonhydro_Capita_2021': 'Wealthy'})

# Append the target column "Wealthy"
SDGs_Scores_2021 = pd.concat([SDGs_Scores_2021, Res_Nonhydro_Capita_2021], axis=1)

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

X = SDGs_Scores_2021.drop('Wealthy',1)
y = SDGs_Scores_2021['Wealthy']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [16]:
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

maxComponents = [ 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
list_n_neighbors = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

n = 0
for componentsi in maxComponents:
    pca = PCA(n_components=componentsi, random_state=42)
    pca.fit(X_train_scaled)
    X_train_scaled_pca = pca.transform(X_train_scaled)
    X_test_scaled_pca = pca.transform(X_test_scaled)
    for n_neighborsi in list_n_neighbors:
        kn_clas = KNeighborsClassifier(n_neighbors = n_neighborsi)
        kn_clas.fit(X_train_scaled_pca, y_train)
        predictions = kn_clas.predict(X_test_scaled_pca)
        if (n == 0): # The first time initialize
            Accuracy_Best = accuracy_score(y_test, predictions)
            n_neighbors_Best = n_neighborsi
            max_components_Best = componentsi
        else:
            Accuracy_New = accuracy_score(y_test, predictions)
            if ( Accuracy_Best < Accuracy_New):
                Accuracy_Best = Accuracy_New
                n_neighbors_Best = n_neighborsi
                max_components_Best = componentsi
            else:
                pass
        n +=1
print('Accuracy_Best', Accuracy_Best, 
        'max_components_Best', max_components_Best,
        'n_estimators', n_neighbors_Best)

Accuracy_Best 0.8214285714285714 max_components_Best 1 n_estimators 3


In [18]:
pipeline = Pipeline(steps = [
    ("scaler", StandardScaler()), # primero escalo
    ("pca", PCA()), # segundo aplica PCA 
    ("kneighborsclassifier", KNeighborsClassifier()) # Despues un KNeighborsClassifier
])

pipeline_param = {
    'pca__n_components' :  [1],
    'pca__random_state' :  [42],
    'kneighborsclassifier__n_neighbors' :  [3]
}

gs_pipeline = GridSearchCV(pipeline,
                            pipeline_param,
                            cv = 10,
                            scoring = 'accuracy',
                            verbose = 1, # mensajes del output
                            n_jobs = -1)

search = GridSearchCV(pipeline, pipeline_param, cv=5).fit(X_train, y_train)

print("Train: Coeficiente de determinacion de la predicción:", search.best_estimator_.score(X_train, y_train))
print("Test: Coeficiente de determinacion de la predicción:", search.best_estimator_.score(X_test, y_test))

Train: Coeficiente de determinacion de la predicción: 0.8807339449541285
Test: Coeficiente de determinacion de la predicción: 0.8214285714285714


# ABAJO ANTOGUO

In [35]:
pipeline = Pipeline(steps = [
    ("scaler", StandardScaler()), # primero escalo
    ("pca", PCA()), # segundo aplica PCA 
    ("kneighborsclassifier", KNeighborsClassifier()) # Despues un KNeighborsClassifier
])

pipeline_param = {
    'pca__n_components' :  list(range(1,17)),
    'kneighborsclassifier__n_neighbors' :  list(range(1,50))
}

gs_pipeline = GridSearchCV(pipeline,
                            pipeline_param,
                             cv = 10,
                            scoring = 'accuracy',
                            verbose = 1, # mensajes del output
                            n_jobs = -1)

grids = {"gs_pipeline": gs_pipeline}

for nombre, grid_search in grids.items():
    grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 784 candidates, totalling 7840 fits


In [36]:
print(gs_pipeline.best_score_)
print(gs_pipeline.best_params_)
print(gs_pipeline.best_estimator_)

0.889090909090909
{'kneighborsclassifier__n_neighbors': 11, 'pca__n_components': 2}
Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=11))])


In [37]:
print("Accuracy train", gs_pipeline.best_estimator_.score(X_train, y_train))
print("Accuracy test", gs_pipeline.best_estimator_.score(X_test, y_test))

Accuracy train 0.8807339449541285
Accuracy test 0.8214285714285714


## MEJOR MODEL
* n_neighbors 11
* n_components 2
* Accuracy train 0.8807339449541285
* Accuracy test 0.8214285714285714

In [187]:
import pickle

with open('model_v1', 'wb') as archivo_salida:
    pickle.dump(gs_pipeline.best_estimator_, archivo_salida)

In [188]:
with open('model_v1', 'rb') as archivo_entrada:
    loaded_model = pickle.load(archivo_entrada)

In [189]:
loaded_model.score(X_test,y_test)

0.8214285714285714

In [171]:
test_csv = pd.concat([X_test, y_test], axis=1)
test_csv

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,SDG11,SDG12,SDG13,SDG14,SDG15,SDG16,SDG17,Wealthy
Country,,,,,,,,,,,,,,,,,,
Portugal,99.3815,64.104500,90.954769,99.003333,83.272750,84.13680,95.022667,82.155333,75.417667,69.697000,88.422000,63.056500,75.819667,43.065000,72.673000,88.067700,62.678750,1
Poland,99.4655,68.159875,85.269923,98.298667,72.180000,84.91120,89.265667,84.405333,71.760000,75.428500,82.428000,75.869333,75.097000,65.122000,92.320000,81.448200,62.381000,1
Belize,55.3460,59.362571,72.339308,77.694333,56.378250,75.87380,92.313500,58.470200,22.748000,36.501007,79.086000,72.233000,91.131667,67.917600,54.174400,60.812125,71.505000,1
Central African Republic,0.0000,43.645500,16.026071,9.207000,30.888250,45.51140,12.827500,47.698000,3.526000,9.641000,19.243000,72.737667,99.673000,48.160239,89.913800,44.737000,37.324333,0
Turkey,99.6240,65.068000,82.928000,96.273000,46.329750,75.12840,89.161000,59.060667,60.301500,38.105000,73.987500,79.810833,86.967667,51.998167,53.259400,66.496444,71.963250,1
Italy,98.0600,71.018625,94.043000,96.708333,72.829000,82.13500,93.847333,76.572000,83.604500,71.545000,82.341250,63.609667,76.820000,53.541167,80.221400,78.989500,62.952750,1
Ireland,99.7950,75.894875,93.231846,99.738708,76.390250,87.28540,93.429333,87.328600,82.017000,86.423000,88.755250,45.179000,58.196333,71.895167,89.083000,90.434333,51.307750,1
Sri Lanka,91.5440,62.129625,77.814571,97.413667,52.004250,56.62700,66.947667,76.219500,34.159167,36.333000,82.970667,85.360167,95.152000,71.716500,55.946600,66.226400,49.136000,0
Vietnam,95.1425,71.032625,72.467429,97.107667,72.203250,73.13800,83.200333,72.149167,43.975667,70.499500,80.116250,91.899667,93.970000,49.192167,46.689000,65.960000,59.681250,0


In [172]:
test_csv.to_csv('test.csv', index = True)

In [173]:
import json
  
# Data to be written
dictionary ={
    "nombre_alumno" : "Miguel Chamochin",
    "titulo" : "Conectando los Objetivos de Desarrollo Sostenible con el cambio climático y la transición energética",
    "tipo_ml" : "O",
    "target" : "Wealthy"
}
  
with open("info.json", "w") as outfile:
    json.dump(dictionary, outfile)

# Predicciones - SDGs_Ranks_2021

In [190]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

X = SDGs_Ranks_2021.drop('Wealthy',1)
y = SDGs_Ranks_2021['Wealthy']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [191]:
pipeline = Pipeline(steps = [
    ("scaler", StandardScaler()), # primero escalo
    ("pca", PCA()), # segundo aplica PCA 
    ("kneighborsclassifier", KNeighborsClassifier()) # Despues un KNeighborsClassifier
])

pipeline_param = {
    'pca__n_components' :  list(range(1,17)),
    'kneighborsclassifier__n_neighbors' :  list(range(1,50))
}

gs_pipeline = GridSearchCV(pipeline,
                            pipeline_param,
                             cv = 10,
                            scoring = 'accuracy',
                            verbose = 1, # mensajes del output
                            n_jobs = -1)

grids = {"gs_pipeline": gs_pipeline}

for nombre, grid_search in grids.items():
    grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 784 candidates, totalling 7840 fits


In [192]:
print(gs_pipeline.best_score_)
print(gs_pipeline.best_params_)
print(gs_pipeline.best_estimator_)

0.8890909090909093
{'kneighborsclassifier__n_neighbors': 1, 'pca__n_components': 10}
Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=10)),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=1))])


In [193]:
print("Accuracy train", gs_pipeline.best_estimator_.score(X_train, y_train))
print("Accuracy test", gs_pipeline.best_estimator_.score(X_test, y_test))

Accuracy train 1.0
Accuracy test 0.6785714285714286


# CONCLUSION